<a href="https://colab.research.google.com/gist/SK124/536bf7adcf276f07aad012e5c79ff871/de-novo-drug-synthesis-using-recurrent-neural-networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Following work is an Impementaion of the [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5836943/) "Generative Recurrent Networks for De Novo Drug Design" - Prof [Schneider Gisbert ](https://www.ncbi.nlm.nih.gov/pubmed/?term=Schneider%20G%5BAuthor%5D&cauthor=true&cauthor_uid=29095571)    et al.###

In this approach we are going to leverage on the beneifts of LSTM RNN Units to generate possible drug sequences for fragment‐based drug discovery.

NECCESARY IMPORTS FOR PREPROCESSING

In [ ]:
import sys
import numpy as np
import pandas as pd
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


This dataframe consisits of real life drug design sequences which we will use to create our very own language model.
The drug sequence is representated in SMILES format. which stands for Simplified Molecular Input Line Entry System. You can read more about SMILES nomenclature [here](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system). 

In [ ]:
df=pd.read_csv('/content/full_data_final.csv')
df.head()

,col
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...


* Make a text file including only sequences of size between 70 and 30, as this boundary captures almost all relevant molecules with which we can learn the SMILES syntax tight and train with ease. This boundary gives us the least trade-off between the two.
 
 
* Token 'G' has been added as start token and Token 'E' is added as End token to denote start point and end point of the all sequences. 
 
* Token 'A' is the padding token since not all the sequences are of size 70.

In [ ]:
lengths=[]

with open('drug.txt','w') as f:
    for i in range(len(df)):
        text=df['col'][i]
       # if (len(text)<70 and len(text)>30):
        lengths.append(len(text))
        t=109-len(text)
            text='G'+text+'E'
            text=text+'A'*t
            f.write(text+'\n')
           # df['col'][i]=text
           # index.append(i)
        
    
    
len(lengths)

In [ ]:
lengths=[]

with open('drug.txt','w') as f:
    for i in range(len(df)):
        text=df['col'][i]
        if (len(text)<70 and len(text)>30):
            lengths.append(len(text))
            t=69-len(text)
            text='G'+text+'E'
            text=text+'A'*t
            f.write(text+'\n')
           # df['col'][i]=text
           # index.append(i)
        
    
    
len(lengths)

92967

Once we have the text file, we load it as a single string for further preprocessing

In [ ]:
raw_texts=''
string=''
with open('drug.txt','r') as f:

  text=f.readlines()
  for tex in text:
    tex=tex.strip('\n')
    string+=tex
    string=str(string)
    
  raw_texts=string
 

In [ ]:
raw_texts

* unique_chars will contain all the unique tokens
char_to_int and int_to_char will contain mappings of tokens with respect to integer and vice versa

* We are popping index 15 here from unique_chars list because it has the padding token 'A' included, I wish to give the padding token index 0 which we will do later.

In [ ]:
unique_chars = sorted(list(set(raw_texts)))
unique_chars.pop(15)
# maps each unique character as int
char_to_int = dict((c, i+1) for i, c in enumerate(unique_chars))


int_to_char = dict((i+1, c) for i, c in enumerate(unique_chars))

In [ ]:
char_to_int

{'#': 1,
 '(': 2,
 ')': 3,
 '+': 4,
 '-': 5,
 '/': 6,
 '1': 7,
 '2': 8,
 '3': 9,
 '4': 10,
 '5': 11,
 '6': 12,
 '7': 13,
 '=': 14,
 '@': 15,
 'B': 16,
 'C': 17,
 'E': 18,
 'F': 19,
 'G': 20,
 'H': 21,
 'I': 22,
 'N': 23,
 'O': 24,
 'P': 25,
 'S': 26,
 '[': 27,
 '\\': 28,
 ']': 29,
 'c': 30,
 'l': 31,
 'n': 32,
 'o': 33,
 'r': 34,
 's': 35}

In [ ]:
mapping_size = len(char_to_int)
reverse_mapping_size = len(int_to_char)
print ("Size of the character to integer dictionary is: ", mapping_size)
print ("Size of the integer to character dictionary is: ", reverse_mapping_size)

Size of the character to integer dictionary is:  35
Size of the integer to character dictionary is:  35


Now we need to obtain our SMILES Sequences from the single string, each seqeunce will have a size of 71 after adding the tokens("G" , "A" , "E" ). we will now load them into a list

In [ ]:
raw=[]
for i in range(len(raw_texts)//71):
  string=raw_texts[71*i:71*(i+1)]
  string=string.strip('A')
  raw.append(string)


In [ ]:
raw[0]

'GCC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1E'

Now we change the SMILES sequences to its integer representaion, we wil load it in a list for further preprocessing

In [ ]:
raw_ints=[[char_to_int[w] for w in drug] for drug in raw]

In [ ]:
raw_ints[0]

[20,
 17,
 17,
 2,
 17,
 3,
 2,
 17,
 3,
 30,
 7,
 30,
 30,
 30,
 8,
 33,
 30,
 30,
 2,
 17,
 17,
 2,
 14,
 24,
 3,
 23,
 30,
 9,
 30,
 30,
 30,
 30,
 30,
 9,
 19,
 3,
 30,
 8,
 30,
 7,
 18]

* Now we split the string sequnece as X and Y
 
* Our model will predict sequence Y from sequence X
 
* As the authors say in the paper " The method for training that we used, as shown in Figure, pads every input string to n tokens, where n is the length of the longest SMILES string. For each token, the model predicts the next token in the sequence"
 
 
 Follow this [link](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5836943/figure/minf201700111-fig-0003/) for the image
 
 
* Sequence length = 71
 
* So,
 
 
 
1.   x = first 70 integer values
2.   y = last 70 integer values

In [ ]:
x=[]
y=[]
for i in range(len(raw_ints)):
  x.append(raw_ints[i][:-1])
  y.append(raw_ints[i][1:])


In [ ]:
len(x[0])

40

In [ ]:
len(y[0])

40

max_words here is timesteps or sequence length which is 70 in our case

In [ ]:
max_words=70


In [ ]:
vocab_size=len(char_to_int)
vocab_size

35

Adding a 1 because we deleted Padding token index from  the list but we need it for integer to string SMILE conversion as it is present in many sequences whose size is less than 70.

In [ ]:
n_chars=vocab_size+1
n_chars

36

One Hot encoing the sequence as our model expects a 3D Tensor, we use numpy here to do it in ease, however you can do it yourself, code attached in sampling part of code for the same

In [ ]:
X = np.array([to_categorical(pad_sequences((sent,), max_words),  
       n_chars)  for sent in x])
X = X[:, 0, :, :]
print(X.shape)

(92967, 70, 36)


In [ ]:
Y = np.array([to_categorical(pad_sequences((sent,), max_words),  
       n_chars)  for sent in y])
Y=Y[:, 0, :, :]
print(Y.shape)

(92967, 70, 36)


Imports for the model

In [ ]:
from keras import backend as K
from keras.layers import Dense, LSTM, TimeDistributed, BatchNormalization
from keras.optimizers import Adam
from keras.models import Sequential, load_model, Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import keras



### MODEL ARCHITECTURE

In [ ]:
model = Sequential()
model.add(BatchNormalization(input_shape=(None, n_chars)))
        
model.add(LSTM(256, unit_forget_bias=True, dropout=0.5, 
                                 return_sequences=True))        
model.add(LSTM(256, unit_forget_bias=True, dropout=0.5, 
                                 return_sequences=True))
model.add(BatchNormalization(input_shape=(None, n_chars)))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, None, 36)          144       
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         300032    
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 256)         1024      
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 36)          9252      
Total params: 835,764
Trainable params: 835,180
Non-trainable params: 584
_________________________________________________________________
None


In [ ]:
mkdir weights

In [ ]:
filepath="/content/weights/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
callbacks_list = [checkpoint]

# # Fit the model # epochs supposed to be 50
model.fit(X, Y, epochs = 5+3, batch_size = 64, callbacks = callbacks_list)

Epoch 1/3
92967/92967 [==============================] - 585s 6ms/step - loss: 0.7045

Epoch 00001: loss improved from inf to 0.70450, saving model to /content/weights/weights-improvement-01-0.7045.hdf5
Epoch 2/3
92967/92967 [==============================] - 589s 6ms/step - loss: 0.6938

Epoch 00002: loss improved from 0.70450 to 0.69378, saving model to /content/weights/weights-improvement-02-0.6938.hdf5
Epoch 3/3
92967/92967 [==============================] - 582s 6ms/step - loss: 0.6856

Epoch 00003: loss improved from 0.69378 to 0.68559, saving model to /content/weights/weights-improvement-03-0.6856.hdf5


Now would be a good time to add back our padding token with index 0 as I wanted.


In [ ]:
int_to_char.update({0:'A'})
char_to_int.update({'A':0})

SAMPLING STRATEGY 

* Sample funcion takes the start token "G", end token "E", temperature a parameter which is explained below,also length of desired seqeunce and mappings between both integer and token to convert back the numbers, the model spits the drug seqeunce in integer form and this function therefore returns us sequence in SMILES format, Our model starts generating sequence after we give it the "G" token.
 
* It will either stop the sequence before it reaches maximum length (max_len) which is a parameter we can tweak or after encountering "E" token which denotes the end of sequence.
 
* It has two helper functions get_token_probas which is better  than the usual argmax we usally do because it brings in variance or temperature as its called in thermodynamics and one_hot_encode function from scratch which we need in sampling.

In [ ]:
def one_hot_encode(token_lists, n_chars):
    
    output = np.zeros((len(token_lists), len(token_lists[0]), n_chars))
    for i, token_list in enumerate(token_lists):
        for j, token in enumerate(token_list):
            output[i, j, int(token)] = 1
    return output
         
def sample(model, temp, start_char, end_char, max_len, indices_token, token_indices):
    
    n_chars = len(indices_token)

    seed_token =  [token_indices[start_char]]
    generated = indices_token[int(str(seed_token[0]))]
    
    
    while generated[-1] != end_char:# and len(generated) < max_len:
        x_seed = one_hot_encode([seed_token], n_chars)
        full_preds = model.predict(x_seed, verbose=0)[0]
        logits = full_preds[-1]
        
        probas, next_char_ind = get_token_proba(logits, temp)
                
        next_char = indices_token[int(str(next_char_ind))]
        generated += next_char
        seed_token += [next_char_ind]

    gen=''   
    for i in generated:
      if i!='A':
        gen+=i   
    return gen

def get_token_proba(preds, temp):
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    
    probas = exp_preds / np.sum(exp_preds)
    char_ind = np.argmax(np.random.multinomial(1, probas, 1))
    
    return probas, char_ind

def softmax(preds):
    return np.exp(preds)/np.sum(np.exp(preds))


Now let us look at few samples our model has generated

In [ ]:
sample(model,0.5,'G','E',70,int_to_char,char_to_int)

'GCOc1ccccc1NC(=O)[C@@H]1CCN(C(=O)[C@@H]2CCC[NH+](Cc3ccccc3)C2)C[C@@H]1C)OE'

In [ ]:
sample(model,0.75,'G','E',70,int_to_char,char_to_int)

'GO=C(NCc1cccc(CN2C(=O)[C@@H](CCCN([C@H]3CCOO4)c2)c2ccccc2)c(=O)[nH]c1=OE'

In [ ]:
sample(model,1,'G','E',70,int_to_char,char_to_int)

'GCc1ccc([C@]2(O)NC(=O)c2ccc(C3=O)Nc4ccccc3)n[(H]32=C[C@@H](c2ccccc2F)cc1E'

We can vary the temperature parameter to see the effect of variance on the generated sequence.

Temperature is a important parameter in sampling strategy. The high temperature sample displays greater linguistic variety, but the low temperature sample is more grammatically correct. Such is the world of temperature sampling - lowering the temperature allows you to focus on higher probability output sequences and smooth over deficiencies of the model.


In [ ]:
sample(model,0.1,'G','E',70,int_to_char,char_to_int)

'GCC(=O)N1CCC(C(=O)N2CCC[C@@H](c3ccccc3)[C@@H]2c2ccccc2)[C@@H]1c1ccccc1E'

In [ ]:
sample(model,0.2,'G','E',70,int_to_char,char_to_int)

'GCC(=O)N1CCC(C(=O)N2CCC[C@@H](c3ccccc3)[C@@H]2c2ccc(Cl)cc2)c2ccccc2[O-]E'

In [ ]:
sample(model,0.3,'G','E',70,int_to_char,char_to_int)

'GCC1=C(C)CC(=O)N2CCC(C(=O)N3CCC[C@@H](C(=O)N4CCCC4)CC3)CC2)c2ccccc2[nH]1E'

In [ ]:
sample(model,0.4,'G','E',70,int_to_char,char_to_int)

'GCC[C@H]1CC(=O)N2CC[C@@H](C(=O)N3CCC(C(=O)Nc4ccccc4)CC3)C2)c2ccccc21E'

In [ ]:
sample(model,0.5,'G','E',70,int_to_char,char_to_int)

'GCOc1ccc(-c2ccccc2)c(C[NH+]2CCC([C@@H]3CCCN3C(=O)Cc3ccc(F)cc3)CC2=O)c1E'

In [ ]:
sample(model,0.6,'G','E',70,int_to_char,char_to_int)

'GCC1=C(NC(=O)[C@H]2CCCN2C(=O)C2CCO(C(=O)c3ccccc4)CC3)C2)C[C@@H]1C(=O)OCE'

In [ ]:
sample(model,0.7,'G','E',70,int_to_char,char_to_int)

'GCC(=O)c1ccc2c(c1)=C(=O)C1=C(C)c3ccccc3=[NH+][C@@H]3c1ccccc3)cc(Cl)c2c1E'

In [ ]:
sample(model,0.8,'G','E',70,int_to_char,char_to_int)

'GCC[C@H]1CCCC[C@@H]1NC(=O)[C@@H]1CC(=O)N(C(C)(C)C)CC1)[C@@H](C)c1ccccc1E'

In [ ]:
sample(model,0.9,'G','E',70,int_to_char,char_to_int)

'GCC1(C)[C@H]2[NH3+])C[C@H](CN2C(=O)Nc3ccccc3)C[C@@H]2[NH+](CC[C@H]3C)C1E'

In [ ]:
sample(model,1.0,'G','E',70,int_to_char,char_to_int)

'GCOc1cc(O[C@@H](C)Nc2nc(CC(=O)N3C[C@@H](C)O[C@H](c5ccccn4)C3)co2)ccc1OE'

* I guess, we can safely say our model really makes us convince that these molecules actually exist in nature!
Most of the times it  gets all the syntax of SMILES right, with just 8 epochs ( 1.5 hr on Google Colab GPU : Tesla K80) like negative charge after Oxygen, positive charge with NH2, opening and closing brackets after starting a subsequence within a sequence.
 
* We can train this model for a much longer time and we can expect more real like sequences. I'm told that authors have trained their network for 21 Hrs! albeit they had a much bigger datatset compared to what we have therefore they used a bigger network with more parameters as well, hence longer time to train.

Acknowldegements:

Huge thanks to Prof Dr [Schenider Gisbert](https://cadd.ethz.ch/people/gisbert_Schneider.html) , Researcher in Computer-assisted drug design at ETH Zürich, Switzerland for providing access to his papers for better understanding of his research work without which this work would not have been possible.


**************************************************************************************************************